In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob

import pandas as pd

In [3]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': 'C:\\AgodaGit\\passivbot\\' + filename}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        #d['adgt_%'] = d['adgl_%']+d['adgs_%']
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_233100/1315120660.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_233100/1315120660.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_233100/1315120660.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_233100/1315120660.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%
0,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.909573,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.114573,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.838780,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.952475,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.242079,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,0.841806,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
743,C:\AgodaGit\passivbot\backtests/binance\ZECUSD...,5.4,ZECUSDT,0.966600,True,True,0.0600,0.0179,100.0,100.0,0.9792,12.53,3.573
744,C:\AgodaGit\passivbot\backtests/binance\ZECUSD...,5.4,ZECUSDT,0.966600,True,True,0.0600,0.0179,100.0,100.0,0.9792,12.53,3.573
745,C:\AgodaGit\passivbot\backtests/binance\ZECUSD...,5.5,ZECUSDT,0.966000,True,True,0.0599,0.0173,100.0,100.0,0.9792,12.52,3.435


Now filter the dataframe

In [4]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = ["LINAUSDT","CRVUSDT","LINKUSDT","MATICUSDT","ALGOUSDT","ENJUSDT"]
big_data['adgt_%'] = big_data['adgl_%']+big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
              & (big_data['long'] == True)
              & (big_data['short'] == True)
              #& (big_data['adgt_%'] > 0.1 )
              & (big_data['bnkrl_%'] > 95.0)
              & (big_data['bnkrs_%'] > 95.0)
              & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%,adgt_%
63,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,5.5,ALGOUSDT,0.9478,True,True,0.0682,0.0501,100.0,100.0,0.8697,15.440,9.608,0.1183
64,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,5.5,ALGOUSDT,0.9478,True,True,0.0682,0.0501,100.0,100.0,0.8697,15.440,9.608,0.1183
65,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,5.5,ALGOUSDT,0.9624,True,True,0.0580,0.0569,100.0,100.0,0.8823,12.500,11.110,0.1149
66,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,5.5,ALGOUSDT,0.9478,True,True,0.0682,0.0501,100.0,100.0,0.8697,15.440,9.608,0.1183
67,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,5.5,ALGOUSDT,0.9624,True,True,0.0580,0.0569,100.0,100.0,0.8823,12.500,11.110,0.1149
68,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,5.5,ALGOUSDT,0.9478,True,True,0.0682,0.0501,100.0,100.0,0.8697,15.440,9.608,0.1183
69,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,5.5,ALGOUSDT,0.9361,True,True,0.0943,0.0832,100.0,100.0,0.8720,22.220,19.430,0.1775
240,C:\AgodaGit\passivbot\backtests/binance\CRVUSD...,5.5,CRVUSDT,0.9682,True,True,0.0449,0.0569,100.0,100.0,0.9440,9.286,12.010,0.1018
241,C:\AgodaGit\passivbot\backtests/binance\CRVUSD...,5.5,CRVUSDT,0.9759,True,True,0.0443,0.0610,100.0,100.0,0.9651,9.168,12.860,0.1053
242,C:\AgodaGit\passivbot\backtests/binance\CRVUSD...,5.5,CRVUSDT,0.9682,True,True,0.0449,0.0562,100.0,100.0,0.9656,9.288,11.700,0.1011


In [5]:
df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
dx = df[['file','s','adgt_%','adgl_%','adgs_%','bnkrl_%', 'bnkrs_%','ebrl', 'ebrs']]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_233100/2043545472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',


,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs
63,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,ALGOUSDT,0.1183,0.0682,0.0501,100.0,100.0,0.9478,0.8697
65,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,ALGOUSDT,0.1149,0.0580,0.0569,100.0,100.0,0.9624,0.8823
69,C:\AgodaGit\passivbot\backtests/binance\ALGOUS...,ALGOUSDT,0.1775,0.0943,0.0832,100.0,100.0,0.9361,0.8720
240,C:\AgodaGit\passivbot\backtests/binance\CRVUSD...,CRVUSDT,0.1018,0.0449,0.0569,100.0,100.0,0.9682,0.9440
241,C:\AgodaGit\passivbot\backtests/binance\CRVUSD...,CRVUSDT,0.1053,0.0443,0.0610,100.0,100.0,0.9759,0.9651
242,C:\AgodaGit\passivbot\backtests/binance\CRVUSD...,CRVUSDT,0.1011,0.0449,0.0562,100.0,100.0,0.9682,0.9656
246,C:\AgodaGit\passivbot\backtests/binance\CRVUSD...,CRVUSDT,0.1473,0.0798,0.0675,100.0,100.0,0.9257,0.7864
300,C:\AgodaGit\passivbot\backtests/binance\ENJUSD...,ENJUSDT,0.1171,0.0580,0.0591,100.0,100.0,0.9333,0.9715
302,C:\AgodaGit\passivbot\backtests/binance\ENJUSD...,ENJUSDT,0.1926,0.1010,0.0916,100.0,100.0,0.9288,0.9010
444,C:\AgodaGit\passivbot\backtests/binance\LINAUS...,LINAUSDT,0.0882,0.0361,0.0521,100.0,100.0,0.9763,0.9629
